In [1]:
import pandas as pd
import json

# 1.处理inter

In [2]:
name = "movie"

train_path = f"../../resource/{name}/{name}_ft.json"
test_path = f"../../resource/{name}/{name}_pd.json"

w_train_path = f"../../llm/{name}/{name}_ft.json"
w_test_path = f"../../llm/{name}{name}_pd.json"

In [3]:
# 读数据
inter_data = pd.read_csv(inter_path, sep="\t").sort_values(by=['user_id:token', 'timestamp:float'], ascending=[True, True])
user_data = pd.read_csv(user_path, sep="\t").sort_values(by=['user_id:token'], ascending=[True])
item_data = pd.read_csv(item_path, sep="\t").sort_values(by=['item_id:token'], ascending=[True])
inter_data,user_data,item_data

(       user_id:token  item_id:token  rating:float  timestamp:float
 59972              1            168             5        874965478
 92487              1            172             5        874965478
 74577              1            165             5        874965518
 48214              1            156             4        874965556
 15764              1            196             5        874965677
 ...              ...            ...           ...              ...
 93377            943            449             1        888693158
 94914            943            229             2        888693158
 95003            943            230             1        888693158
 95291            943            228             3        888693158
 92880            943            234             3        888693184
 
 [100000 rows x 4 columns],
      user_id:token  age:token gender:token occupation:token zip_code:token
 0                1         24            M       technician          85711
 1

In [4]:
# 分组&聚合
data_new = inter_data
data_u = data_new.groupby(['user_id:token']).count()
data_u["count"] = data_u["item_id:token"]
# data_u = data_u.drop(['item_id:token','num_repeat:float','timestamp:float'], axis=1)

# 筛选=>10 core
data_u = data_u[data_u["count"] >= 10]

# 按user_id筛选
inter_data = data_new[data_new["user_id:token"].isin(data_u.index)]
inter_data

,user_id:token,item_id:token,rating:float,timestamp:float
59972,1,168,5,874965478
92487,1,172,5,874965478
74577,1,165,5,874965518
48214,1,156,4,874965556
15764,1,196,5,874965677
...,...,...,...,...
93377,943,449,1,888693158
94914,943,229,2,888693158
95003,943,230,1,888693158
95291,943,228,3,888693158


In [5]:
# 按user_id分组，并将其他字段存为列表
inter_data = inter_data.groupby('user_id:token').agg(list).reset_index()
inter_data

,user_id:token,item_id:token,rating:float,timestamp:float
0,1,"[168, 172, 165, 156, 196, 166, 187, 14, 250, 1...","[5, 5, 5, 4, 5, 5, 4, 5, 4, 5, 3, 5, 5, 5, 5, ...","[874965478, 874965478, 874965518, 874965556, 8..."
1,2,"[286, 258, 305, 307, 288, 312, 301, 292, 303, ...","[4, 3, 3, 3, 3, 3, 4, 4, 4, 1, 4, 4, 4, 4, 3, ...","[888549960, 888549961, 888550065, 888550066, 8..."
2,3,"[300, 302, 333, 344, 268, 337, 303, 345, 354, ...","[2, 2, 2, 4, 3, 1, 3, 3, 3, 2, 2, 2, 1, 2, 2, ...","[889236939, 889236939, 889236939, 889236939, 8..."
3,4,"[258, 288, 300, 328, 271, 303, 329, 327, 359, ...","[5, 4, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[892001374, 892001445, 892001445, 892001537, 8..."
4,5,"[267, 222, 455, 121, 363, 405, 257, 250, 25, 2...","[4, 4, 4, 4, 3, 3, 5, 3, 3, 3, 5, 5, 1, 4, 3, ...","[875635064, 875635174, 875635174, 875635189, 8..."
...,...,...,...,...
938,939,"[689, 326, 890, 266, 680, 258, 127, 9, 257, 27...","[5, 5, 2, 2, 2, 4, 5, 5, 5, 4, 4, 5, 5, 5, 5, ...","[880260636, 880260636, 880260636, 880260636, 8..."
939,940,"[286, 319, 310, 683, 301, 347, 271, 264, 315, ...","[3, 2, 3, 3, 3, 3, 2, 1, 4, 3, 3, 1, 2, 4, 5, ...","[884800898, 884800944, 884800966, 884800988, 8..."
940,941,"[258, 300, 294, 358, 117, 408, 181, 298, 919, ...","[4, 4, 4, 2, 5, 5, 5, 5, 5, 4, 4, 5, 4, 3, 4, ...","[875048495, 875048495, 875048532, 875048581, 8..."
941,942,"[315, 750, 347, 313, 269, 310, 362, 272, 258, ...","[4, 4, 5, 3, 2, 4, 3, 5, 4, 5, 4, 3, 3, 4, 5, ...","[891282355, 891282355, 891282396, 891282396, 8..."


In [6]:
# 拼接用户信息 json
gender=dict({'M':'male','F':'female'})
user_data['info'] = user_data.apply(lambda row: dict({"user_id":str(row["user_id:token"]),"age":str(row["age:token"]),"gender":gender[row["gender:token"]],"occupation":row["occupation:token"],"zip_code":row["zip_code:token"]}), axis=1)
user_data['info'].iloc[0]

{'user_id': '1',
 'age': '24',
 'gender': 'male',
 'occupation': 'technician',
 'zip_code': '85711'}

In [7]:
inter_data

,user_id:token,item_id:token,rating:float,timestamp:float
0,1,"[168, 172, 165, 156, 196, 166, 187, 14, 250, 1...","[5, 5, 5, 4, 5, 5, 4, 5, 4, 5, 3, 5, 5, 5, 5, ...","[874965478, 874965478, 874965518, 874965556, 8..."
1,2,"[286, 258, 305, 307, 288, 312, 301, 292, 303, ...","[4, 3, 3, 3, 3, 3, 4, 4, 4, 1, 4, 4, 4, 4, 3, ...","[888549960, 888549961, 888550065, 888550066, 8..."
2,3,"[300, 302, 333, 344, 268, 337, 303, 345, 354, ...","[2, 2, 2, 4, 3, 1, 3, 3, 3, 2, 2, 2, 1, 2, 2, ...","[889236939, 889236939, 889236939, 889236939, 8..."
3,4,"[258, 288, 300, 328, 271, 303, 329, 327, 359, ...","[5, 4, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[892001374, 892001445, 892001445, 892001537, 8..."
4,5,"[267, 222, 455, 121, 363, 405, 257, 250, 25, 2...","[4, 4, 4, 4, 3, 3, 5, 3, 3, 3, 5, 5, 1, 4, 3, ...","[875635064, 875635174, 875635174, 875635189, 8..."
...,...,...,...,...
938,939,"[689, 326, 890, 266, 680, 258, 127, 9, 257, 27...","[5, 5, 2, 2, 2, 4, 5, 5, 5, 4, 4, 5, 5, 5, 5, ...","[880260636, 880260636, 880260636, 880260636, 8..."
939,940,"[286, 319, 310, 683, 301, 347, 271, 264, 315, ...","[3, 2, 3, 3, 3, 3, 2, 1, 4, 3, 3, 1, 2, 4, 5, ...","[884800898, 884800944, 884800966, 884800988, 8..."
940,941,"[258, 300, 294, 358, 117, 408, 181, 298, 919, ...","[4, 4, 4, 2, 5, 5, 5, 5, 5, 4, 4, 5, 4, 3, 4, ...","[875048495, 875048495, 875048532, 875048581, 8..."
941,942,"[315, 750, 347, 313, 269, 310, 362, 272, 258, ...","[4, 4, 5, 3, 2, 4, 3, 5, 4, 5, 4, 3, 3, 4, 5, ...","[891282355, 891282355, 891282396, 891282396, 8..."


In [8]:
user_data

,user_id:token,age:token,gender:token,occupation:token,zip_code:token,info
0,1,24,M,technician,85711,"{'user_id': '1', 'age': '24', 'gender': 'male'..."
1,2,53,F,other,94043,"{'user_id': '2', 'age': '53', 'gender': 'femal..."
2,3,23,M,writer,32067,"{'user_id': '3', 'age': '23', 'gender': 'male'..."
3,4,24,M,technician,43537,"{'user_id': '4', 'age': '24', 'gender': 'male'..."
4,5,33,F,other,15213,"{'user_id': '5', 'age': '33', 'gender': 'femal..."
...,...,...,...,...,...,...
938,939,26,F,student,33319,"{'user_id': '939', 'age': '26', 'gender': 'fem..."
939,940,32,M,administrator,02215,"{'user_id': '940', 'age': '32', 'gender': 'mal..."
940,941,20,M,student,97229,"{'user_id': '941', 'age': '20', 'gender': 'mal..."
941,942,48,F,librarian,78209,"{'user_id': '942', 'age': '48', 'gender': 'fem..."


In [9]:
# 拼接user信息, json
gender = dict({'M':'male','F':'female'})
user_data['info'] = user_data.apply(lambda row: dict({"user_id":row["user_id:token"],"age":row["age:token"],"gender":gender[row["gender:token"]],"occupation":row["gender:token"],"zip_code":row["zip_code:token"]}), axis=1)
user_data['info'].iloc[0]

{'user_id': 1,
 'age': 24,
 'gender': 'male',
 'occupation': 'M',
 'zip_code': '85711'}

In [10]:
item_data

,item_id:token,movie_title:token_seq,release_year:token,class:token_seq
0,1,Toy Story,1995,Animation Children's Comedy
1,2,GoldenEye,1995,Action Adventure Thriller
2,3,Four Rooms,1995,Thriller
3,4,Get Shorty,1995,Action Comedy Drama
4,5,Copycat,1995,Crime Drama Thriller
...,...,...,...,...
1677,1678,Mat' i syn,1997,Drama
1678,1679,B. Monkey,1998,Romance Thriller
1679,1680,Sliding Doors,1998,Drama Romance
1680,1681,You So Crazy,1994,Comedy


In [11]:
# 拼接item信息, json
item_data['info'] = item_data.apply(lambda row: dict({"item_id":row["item_id:token"],"movie_title":row["movie_title:token_seq"],"release_year":row["release_year:token"],"classification":row["class:token_seq"],"rating":-1}), axis=1)
item_data['info'].iloc[0]

{'item_id': 1,
 'movie_title': 'Toy Story',
 'release_year': '1995',
 'classification': "Animation Children's Comedy",
 'rating': -1}

In [12]:
def build_item(i,r):
    item = (item_data[item_data['item_id:token'] == i]['info'].iloc[0])
    item['rating'] = r
    return item

In [13]:
def build_inter(row):
    user = user_data[user_data['user_id:token'] == row['user_id:token']]['info'].iloc[0]
    items = [build_item(i,r) for i,r in zip(row['item_id:token'],row['rating:float'])]
    val = dict({'user':user,'items':items})
    return val

In [14]:
# 拼接inter信息, json
inter_data['info'] = inter_data.apply(lambda row: build_inter(row), axis=1)
inter_data['info'].iloc[0]

{'user': {'user_id': 1,
  'age': 24,
  'gender': 'male',
  'occupation': 'M',
  'zip_code': '85711'},
 'items': [{'item_id': 168,
   'movie_title': 'Monty Python and the Holy Grail',
   'release_year': '1974',
   'classification': 'Comedy',
   'rating': 2},
  {'item_id': 172,
   'movie_title': 'Empire Strikes Back, The',
   'release_year': '1980',
   'classification': 'Action Adventure Drama Romance Sci-Fi War',
   'rating': 4},
  {'item_id': 165,
   'movie_title': 'Jean de Florette',
   'release_year': '1986',
   'classification': 'Drama',
   'rating': 4},
  {'item_id': 156,
   'movie_title': 'Reservoir Dogs',
   'release_year': '1992',
   'classification': 'Crime Thriller',
   'rating': 3},
  {'item_id': 196,
   'movie_title': 'Dead Poets Society',
   'release_year': '1989',
   'classification': 'Drama',
   'rating': 5},
  {'item_id': 166,
   'movie_title': 'Manon of the Spring (Manon des sources)',
   'release_year': '1986',
   'classification': 'Drama',
   'rating': 3},
  {'item_id

In [15]:
def cal_label(rating):
    return "Yes" if rating > threshold else "No"
    
inter_temp = '''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
Predict the user's rating for the target movie based on their basic information and historical ratings. Returns only whether the result is greater than {threshold}. For example Yes or No.

### Input:
User Information:
{user_information}

History Ratings:
{history_ratings}

Target Movie:
{target_movie}

### Response:
{label}'''

max_length = 50
threshold = 3

def build_prompt(row,data_type):
    user = user_data[user_data['user_id:token'] == row['user_id:token']]['info'].iloc[0]
    items = [build_item(i,r) for i,r in zip(row['item_id:token'],row['rating:float'])]
    
    length = max_length
    if len(items)<max_length:
        length = len(items)
    
    user_information_json = user
    history_ratings_json = items[:(length-3)]
    train_target_movie_json = items[length-3]
    valid_target_movie_json = items[length-2]
    test_target_movie_json = items[length-1]

    train_label = train_target_movie_json.pop("rating")
    valid_label = valid_target_movie_json.pop("rating")
    test_label = test_target_movie_json.pop("rating")

    temp = inter_temp.replace('{threshold}',str(threshold)).replace('{user_information}',json.dumps(user)).replace('{history_ratings}',json.dumps(history_ratings_json))
    train_val = temp.replace('{target_movie}',json.dumps(train_target_movie_json)).replace('{label}','Yes' if train_label>threshold else 'No')
    valid_val = temp.replace('{target_movie}',json.dumps(valid_target_movie_json)).replace('{label}','Yes' if valid_label>threshold else 'No')
    test_val = temp.replace('{target_movie}',json.dumps(test_target_movie_json)).replace('{label}','Yes' if test_label>threshold else 'No')

    if data_type=='train':
        val = {'prompt': train_val, 'label': train_label}
    elif data_type=='valid':
        val = {'prompt': valid_val, 'label': valid_label}
    elif data_type=='test':
        val = {'prompt': test_val, 'label': test_label}
    
    return val

In [16]:
# v0.1
def cal_label(rating):
    return "1" if rating > threshold else "0"
    
inter_temp = '''Predict the user's rating for the target movie based on their basic information and historical ratings. Returns only whether the result is greater than {threshold}. For example 1 or 0. # noqa: E501

User Information:
{user_information}

Target Movie:
{target_movie}

History Ratings:
{history_ratings}
'''

max_length = 50
threshold = 3

def build_prompt(row):
    user = user_data[user_data['user_id:token'] == row['user_id:token']]['info'].iloc[0]
    items = [build_item(i,r) for i,r in zip(row['item_id:token'],row['rating:float'])]
    
    length = max_length
    if len(items)<max_length:
        length = len(items)
    
    user_information_json = user
    history_ratings_json = items[:(length-3)]
    train_target_movie_json = items[length-3]
    valid_target_movie_json = items[length-2]
    test_target_movie_json = items[length-1]

    train_label = train_target_movie_json.pop("rating")
    valid_label = valid_target_movie_json.pop("rating")
    test_label = test_target_movie_json.pop("rating")

    temp = inter_temp.replace('{threshold}',str(threshold)).replace('{user_information}',json.dumps(user)).replace('{history_ratings}',json.dumps(history_ratings_json))
    train_val = temp.replace('{target_movie}',json.dumps(train_target_movie_json))
    valid_val = temp.replace('{target_movie}',json.dumps(valid_target_movie_json))
    test_val = temp.replace('{target_movie}',json.dumps(test_target_movie_json))

    info = {}
    info['train'] = {'prompt': train_val, 'label': cal_label(train_label)}
    info['valid'] = {'prompt': valid_val, 'label': cal_label(valid_label)}
    info['test'] = {'prompt': test_val, 'label': cal_label(test_label)}
    
    return info

In [15]:
# v0.2
def cal_label(rating):
    return "Yes" if rating > threshold else "No"
    
inter_temp = '''Predict the user's rating for the target movie based on their basic information and historical ratings. Returns only whether the result is greater than {threshold}. For example Yes or No. # noqa: E501

User Information:
{user_information}

Target Movie:
{target_movie}

History Ratings:
{history_ratings}
'''

max_length = 50
threshold = 3

def build_prompt(row):
    user = user_data[user_data['user_id:token'] == row['user_id:token']]['info'].iloc[0]
    items = [build_item(i,r) for i,r in zip(row['item_id:token'],row['rating:float'])]
    
    length = max_length
    if len(items)<max_length:
        length = len(items)
    
    user_information_json = user
    history_ratings_json = items[:(length-3)]
    train_target_movie_json = items[length-3]
    valid_target_movie_json = items[length-2]
    test_target_movie_json = items[length-1]

    train_label = train_target_movie_json.pop("rating")
    valid_label = valid_target_movie_json.pop("rating")
    test_label = test_target_movie_json.pop("rating")

    temp = inter_temp.replace('{threshold}',str(threshold)).replace('{user_information}',json.dumps(user)).replace('{history_ratings}',json.dumps(history_ratings_json))
    train_val = temp.replace('{target_movie}',json.dumps(train_target_movie_json))
    valid_val = temp.replace('{target_movie}',json.dumps(valid_target_movie_json))
    test_val = temp.replace('{target_movie}',json.dumps(test_target_movie_json))

    info = {}
    info['train'] = {'instruction': train_val, 'input': '', 'output': cal_label(train_label)}
    info['valid'] = {'instruction': valid_val, 'input': '', 'output': cal_label(valid_label)}
    info['test'] = {'instruction': test_val, 'output': cal_label(test_label)}
    
    return info

In [ ]:
# v0.3
def cal_label(rating):
    return "Yes" if rating > threshold else "No"
    
inter_temp = '''Based on the user's basic information and historical information, predict whether the user likes the target movie or not. Returns only whether the result is like Yes or No. # noqa: E501

User Information:
{user_information}

Target Movie:
{target_movie}

Historical Information:
{history_ratings}
'''

max_length = 50
threshold = 3

def build_prompt(row):
    user = user_data[user_data['user_id:token'] == row['user_id:token']]['info'].iloc[0]
    items = [build_item(i,r) for i,r in zip(row['item_id:token'],row['rating:float'])]
    
    length = max_length
    if len(items)<max_length:
        length = len(items)
    
    user_information_json = user
    history_ratings_json = items[:(length-3)]
    train_target_movie_json = items[length-3]
    valid_target_movie_json = items[length-2]
    test_target_movie_json = items[length-1]

    train_label = train_target_movie_json.pop("rating")
    valid_label = valid_target_movie_json.pop("rating")
    test_label = test_target_movie_json.pop("rating")

    temp = inter_temp.replace('{threshold}',str(threshold)).replace('{user_information}',json.dumps(user)).replace('{history_ratings}',json.dumps(history_ratings_json))
    train_val = temp.replace('{target_movie}',json.dumps(train_target_movie_json))
    valid_val = temp.replace('{target_movie}',json.dumps(valid_target_movie_json))
    test_val = temp.replace('{target_movie}',json.dumps(test_target_movie_json))

    info = {}
    info['train'] = {'instruction': train_val, 'input': '', 'output': cal_label(train_label)}
    info['valid'] = {'instruction': valid_val, 'input': '', 'output': cal_label(valid_label)}
    info['test'] = {'instruction': test_val, 'output': cal_label(test_label)}
    
    return info

In [16]:
# 拼接inter信息, json
inter_data['info'] = inter_data.apply(lambda row: build_prompt(row), axis=1)
inter_data['train'] = inter_data.apply(lambda row: row['info']['train'], axis=1)
inter_data['valid'] = inter_data.apply(lambda row: row['info']['valid'], axis=1)
inter_data['test'] = inter_data.apply(lambda row: row['info']['test'], axis=1)

inter_data['train'].iloc[0],inter_data['valid'].iloc[0],inter_data['test'].iloc[0]

({'instruction': 'Predict the user\'s rating for the target movie based on their basic information and historical ratings. Returns only whether the result is greater than 3. For example Yes or No. # noqa: E501\n\nUser Information:\n{"user_id": 1, "age": 24, "gender": "male", "occupation": "M", "zip_code": "85711"}\n\nTarget Movie:\n{"item_id": 190, "movie_title": "Henry V", "release_year": "1989", "classification": "Drama War"}\n\nHistory Ratings:\n[{"item_id": 168, "movie_title": "Monty Python and the Holy Grail", "release_year": "1974", "classification": "Comedy", "rating": 5}, {"item_id": 172, "movie_title": "Empire Strikes Back, The", "release_year": "1980", "classification": "Action Adventure Drama Romance Sci-Fi War", "rating": 5}, {"item_id": 165, "movie_title": "Jean de Florette", "release_year": "1986", "classification": "Drama", "rating": 5}, {"item_id": 156, "movie_title": "Reservoir Dogs", "release_year": "1992", "classification": "Crime Thriller", "rating": 4}, {"item_id":

In [17]:
inter_data

,user_id:token,item_id:token,rating:float,timestamp:float,info,train,valid,test
0,1,"[168, 172, 165, 156, 196, 166, 187, 14, 250, 1...","[5, 5, 5, 4, 5, 5, 4, 5, 4, 5, 3, 5, 5, 5, 5, ...","[874965478, 874965478, 874965518, 874965556, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...
1,2,"[286, 258, 305, 307, 288, 312, 301, 292, 303, ...","[4, 3, 3, 3, 3, 3, 4, 4, 4, 1, 4, 4, 4, 4, 3, ...","[888549960, 888549961, 888550065, 888550066, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...
2,3,"[300, 302, 333, 344, 268, 337, 303, 345, 354, ...","[2, 2, 2, 4, 3, 1, 3, 3, 3, 2, 2, 2, 1, 2, 2, ...","[889236939, 889236939, 889236939, 889236939, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...
3,4,"[258, 288, 300, 328, 271, 303, 329, 327, 359, ...","[5, 4, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[892001374, 892001445, 892001445, 892001537, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...
4,5,"[267, 222, 455, 121, 363, 405, 257, 250, 25, 2...","[4, 4, 4, 4, 3, 3, 5, 3, 3, 3, 5, 5, 1, 4, 3, ...","[875635064, 875635174, 875635174, 875635189, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...
...,...,...,...,...,...,...,...,...
938,939,"[689, 326, 890, 266, 680, 258, 127, 9, 257, 27...","[5, 5, 2, 2, 2, 4, 5, 5, 5, 4, 4, 5, 5, 5, 5, ...","[880260636, 880260636, 880260636, 880260636, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...
939,940,"[286, 319, 310, 683, 301, 347, 271, 264, 315, ...","[3, 2, 3, 3, 3, 3, 2, 1, 4, 3, 3, 1, 2, 4, 5, ...","[884800898, 884800944, 884800966, 884800988, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...
940,941,"[258, 300, 294, 358, 117, 408, 181, 298, 919, ...","[4, 4, 4, 2, 5, 5, 5, 5, 5, 4, 4, 5, 4, 3, 4, ...","[875048495, 875048495, 875048532, 875048581, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...
941,942,"[315, 750, 347, 313, 269, 310, 362, 272, 258, ...","[4, 4, 5, 3, 2, 4, 3, 5, 4, 5, 4, 3, 3, 4, 5, ...","[891282355, 891282355, 891282396, 891282396, 8...",{'train': {'instruction': 'Predict the user's ...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...,{'instruction': 'Predict the user's rating for...


In [18]:
def export_json(data_type,export_path):
    # 将DataFrame导出为JSON格式
    # 将 'name' 列转换为列表
    ls = inter_data[data_type].tolist()
    
    df = pd.DataFrame(ls)
    # 导出为格式化后的 JSON 文件
    df.to_json(export_path, orient='records')

In [19]:
export_json('train',train_path)
export_json('valid',valid_path)
export_json('test',test_path)

In [20]:
# 读取 JSON 文件到 DataFrame
train_df = pd.read_json(train_path)
train_df

,instruction,input,output
0,Predict the user's rating for the target movie...,,Yes
1,Predict the user's rating for the target movie...,,Yes
2,Predict the user's rating for the target movie...,,Yes
3,Predict the user's rating for the target movie...,,No
4,Predict the user's rating for the target movie...,,Yes
...,...,...,...
938,Predict the user's rating for the target movie...,,No
939,Predict the user's rating for the target movie...,,No
940,Predict the user's rating for the target movie...,,Yes
941,Predict the user's rating for the target movie...,,Yes
